<a href="https://colab.research.google.com/github/Spnetic-5/mail_Spam/blob/main/ClassifierModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt
%matplotlib inline

### Data processing using panda library

In [ ]:
dset = pd.read_csv("https://raw.githubusercontent.com/ShubhamPy/Spam-Classifier/master/spam.tsv",sep='\t',names=['Class','Message'])
dset.head(8)

,Class,Message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
5,ham,As per your request 'Melle Melle (Oru Minnamin...
6,spam,WINNER!! As a valued network customer you have...
7,spam,Had your mobile 11 months or more? U R entitle...


### EDA

In [ ]:
dinfo=dset.info()
dinfo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5567 entries, 0 to 5566
Data columns (total 2 columns):
Class      5567 non-null object
Message    5567 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [ ]:
dset.describe()

,Class,Message
count,5567,5567
unique,2,5164
top,ham,"Sorry, I'll call later"
freq,4821,30


As from continuosly performing EDA it leads to start thinking about the features we are going to be using.Here comes the general idea of feature engineering comes. The better your domain knowledge on the data, the better your ability to engineer more features from it. Feature engineering is a very large part of spam detection in general.

Let's make a new column to detect how long the text messages are:

In [ ]:
dset['Length'] = dset['Message'].apply(len)

In [ ]:
dset.head(8)

,Class,Message,Length
0,ham,I've been searching for the right words to tha...,196
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,"Nah I don't think he goes to usf, he lives aro...",61
3,ham,Even my brother is not like to speak with me. ...,77
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,35
5,ham,As per your request 'Melle Melle (Oru Minnamin...,160
6,spam,WINNER!! As a valued network customer you have...,157
7,spam,Had your mobile 11 months or more? U R entitle...,154


In [ ]:
dset.groupby('Class').count()

,Message,Length
Class,,
ham,4821,4821
spam,746,746


### Data Visualization

In [ ]:
dset['Length'].describe()

count    5567.000000
mean       80.449973
std        59.891157
min         2.000000
25%        36.000000
50%        62.000000
75%       122.000000
max       910.000000
Name: Length, dtype: float64

See what we found, A 910 character long message.
Let's use masking to find this message:

In [ ]:
dset[dset['Length']==910]['Message'].iloc[0]

"For me the love should start with attraction.i should feel that I need her every time around me.she should be the first thing which comes in my thoughts.I would start the day and end it with her.she should be there every time I dream.love will be then when my every breath has her name.my life should happen around her.my life will be named to her.I would cry for her.will give all my happiness and take all her sorrows.I will be ready to fight with anyone for her.I will be in love when I will be doing the craziest things for her.love will be when I don't have to proove anyone that my girl is the most beautiful lady on the whole planet.I will always be singing praises for her.love will be when I start up making chicken curry and end up makiing sambar.life will be the most beautiful then.will get every morning and thank god for the day because she is with me.I would like to say a lot..will tell later.."

### Text Pre-processing

In [ ]:
dObject = dset['Class'].values
dObject

array(['ham', 'spam', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [ ]:
dset.loc[dset['Class']=="ham","Class"] = 1

In [ ]:
dset.loc[dset['Class']=="spam","Class"] = 0

In [ ]:
dObject2=dset['Class'].values
dObject2

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
dset.head(8)

,Class,Message,Length
0,1,I've been searching for the right words to tha...,196
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,1,"Nah I don't think he goes to usf, he lives aro...",61
3,1,Even my brother is not like to speak with me. ...,77
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!,35
5,1,As per your request 'Melle Melle (Oru Minnamin...,160
6,0,WINNER!! As a valued network customer you have...,157
7,0,Had your mobile 11 months or more? U R entitle...,154


First removing punctuation. We can just take advantage of Python's built-in string library to get a quick list of all the possible punctuation:

In [ ]:
#clean message from punctuations
def cleanMessage(message):
    nonPunc = [char for char in message if char not in string.punctuation]
    nonPunc = "".join(nonPunc)
    return nonPunc

Tokenization-( process of converting the normal text strings in to a list of tokens(also known as lemmas)).

In [ ]:
dset['Message'] = dset['Message'].apply(cleanMessage)

In [ ]:
dset.head(8)

,Class,Message,Length
0,1,Ive been searching for the right words to than...,196
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,1,Nah I dont think he goes to usf he lives aroun...,61
3,1,Even my brother is not like to speak with me T...,77
4,1,I HAVE A DATE ON SUNDAY WITH WILL,35
5,1,As per your request Melle Melle Oru Minnaminun...,160
6,0,WINNER As a valued network customer you have b...,157
7,0,Had your mobile 11 months or more U R entitled...,154


Now we need to convert each of those messages into a vector the SciKit Learn's algorithm models can work with and machine learning model which we will gonig to  use can understand.

In [ ]:
CV = CountVectorizer(stop_words="english")

In [ ]:
xSet = dset['Message'].values
ySet = dset['Class'].values

### Splitting Train and Test Data

In [ ]:
xSet_train,xSet_test,ySet_train,ySet_test = train_test_split(xSet,ySet,test_size=0.2)

In [ ]:
xSet_train_CV = CV.fit_transform(xSet_train)

### Training a model

With messages represented as vectors, we can finally train our spam/ham classifier. Now we can actually use almost any sort of classification algorithms. For a variety of reasons, the Naive Bayes classifier algorithm is a good choice.

In [ ]:
NB = MultinomialNB()

In [ ]:
NB.fit(xSet_train_CV,ySet_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Data Munging (or Data Wrangling)- which means preparing the data for dedicated purpose, taking the data from its raw state and transforming and mapping into another format.

In [ ]:
xSet_test_CV = CV.transform(xSet_test)

In [ ]:
ySet_predict = NB.predict(xSet_test_CV)

In [ ]:
accuracyScore = accuracy_score(ySet_test,ySet_predict)*100

In [ ]:
print("Prediction Accuracy :",accuracyScore)

Prediction Accuracy : 98.8330341113106


## SpamClassificationApplication

In [ ]:
msg = input("Enter Message: ")
msgInput = CV.transform([msg])
predict = NB.predict(msgInput)
if(predict[0]==0):
    print("------------------------MESSAGE-SENT-[CHECK-SPAM-FOLDER]---------------------------")
else:
    print("---------------------------MESSAGE-SENT-[CHECK-INBOX]------------------------------")

Enter Message: Thanks for your subscription to Ringtone UK your mobile will be charged $5/month Please confirm by replying YES or NO. If you reply NO you will not be charged
------------------------MESSAGE-SENT-[CHECK-SPAM-FOLDER]---------------------------
